As Katherine Kinnaird and I continue our work on the Tedtalks, we have found ourselves drawn to examine more closely the notion of topics, which we both feel have been underexamined in their usage in the humanities. 

Most humanists use an implementation of LDA, which we will probably also use simply to stay in parallel, but at some point in our work, frustrated with my ability to get LDA to work within Python, I picked up Alan Riddell's DARIAH tutorial and drafted an implementation of NMF topic modeling for our corpus. One advantage I noticed right away, in comparing the results to earlier work I had done with Jonathan Goodwin, was what seemed like a much more stable set of word clusters in the algorithmically-derived topics. 

Okay, good, but Kinnaird noticed that stopwords kept creeping into the topics and that raised larger issues about how NMF does what it does and that meant, because she's so thorough, backing up a bit and making sure we understand how NMF works.

What follows is an experiment to understand the shape and nature of the tf matrix, the tfidf matrix, and the output of the `sklearn` NMF algorithm. Some of this is driven by the following essays:

* [Improving the Interpretation of Topic Models][]
* [Practical Topic Finding for Short-Sentence Texts][]

[Improving the Interpretation of Topic Models]: https://medium.com/towards-data-science/improving-the-interpretation-of-topic-models-87fd2ee3847d
[Practical Topic Finding for Short-Sentence Texts]: https://nbviewer.jupyter.org/github/dolaameng/tutorials/blob/master/topic-finding-for-short-texts/topics_for_short_texts.ipynb

To start our adventure, we needed a small set of texts with sufficient overlap that we could later successfully derive topics from them. I set myself the task of creating ten sentences, each of approximately ten words. Careful readers who take the time to read the sentences themselves will, I hope, forgive me for the texts being rather reflexive in nature, but it did seem appropriate given the overall reflexive nature of this task.

In [1]:
# =-=-=-=-=-=-=-=-=-=-=
# The Toy Corpus
# =-=-=-=-=-=-=-=-=-=-= 

sentences = ["Each of these sentences consists of about ten words.",
             "Ten sentence stories and ten word stories were once popular.",
             "Limiting the vocabulary to ten words is difficult.",
             "It is quite difficult to create sentences of just ten words",
             "I need, in fact, variety in the words used.",
             "With all these texts being about texts, there will be few topics.",
             "But I do not want too much variety in the vocabulary.",
             "I want to keep the total vocabulary fairly small.",
             "With a small vocabulary comes a small matrix.",
             "The smaller the matrix the more we will be able to see how things work."]


# =-=-=-=-=-=-=-=-=-=-=
# The Stopwords for this corpus
# =-=-=-=-=-=-=-=-=-=-= 

stopwords = ["a", "about", "all", "and", "be", "being", "but", "do", "each", "few", 
             "how", "i", "in", "is", "it", "more", "much", "not", "of", "once", "the", 
             "there", "these", "to", "too", "want", "we", "were", "will", "with"]

Each text is simply a sentence in a list of strings. Below the texts is the custom stopword list for this corpus. For those curious, there are a total of 102 tokens in the corpus and 30 stopwords. Once the stopwords are applied, 49 tokens remain for a total of 31 words.

In [2]:
# =-=-=-=-=-=
# Clean & Tokenize
# =-=-=-=-=-=

import re
from nltk.tokenize import WhitespaceTokenizer

tokenizer = WhitespaceTokenizer()
# stopwords = re.split('\s+', open('../data/tt_stop.txt', 'r').read().lower())

# Loop to tokenize, stop, and stem (if needed) texts.
tokenized = []
for sentence in sentences:   
    raw = re.sub(r"[^\w\d'\s]+",'', sentence).lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [word for word in tokens if not word in stopwords]
    tokenized.append(stopped_tokens)

    
# =-=-=-=-=-=-=-=-=-=-=
# Re-Assemble Texts as Strings from Lists of Words
# (because this is what sklearn expects)
# =-=-=-=-=-=-=-=-=-=-= 

texts = []
for item in tokenized:
    the_string = ' '.join(item)
    texts.append(the_string)

In [3]:
for text in texts:
    print(text)

sentences consists ten words
ten sentence stories ten word stories popular
limiting vocabulary ten words difficult
quite difficult create sentences just ten words
need fact variety words used
texts texts topics
variety vocabulary
keep total vocabulary fairly small
small vocabulary comes small matrix
smaller matrix able see things work


In [4]:
all_words = ' '.join(texts).split()
print("There are {} tokens representing {} words."
      .format(len(all_words), len(set(all_words))))

There are 49 tokens representing 31 words.


We will explore below the possibility of using the sklearn module's built-in tokenization and stopword abilities, but while I continue to teach myself that functionality, we can move ahead with understanding the vectorization of a corpus. 

There are a lot of ways to turn a series of words into a series of numbers. One of the principle ways of doing so ignores any individuated context for a particular word as we might understand it within the context of a given sentence but simply considers a word in relationship to other words in a text. That is, one way to turn words into numbers is simply to count the words in a text, reducing a text to what is known as a "bag of words." (There's a lot of linguistics and information science that validates this approach, but it will always chafe most humanists.)

If we run our corpus of ten sentences through the `CountVectorizer`, we will get a representation of it as a series of numbers, each representing the count of a particular word within a particular text:

In [5]:
# =-=-=-=-=-=-=-=-=-=-=
# TF
# =-=-=-=-=-=-=-=-=-=-= 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vec = CountVectorizer()
tf_data = vec.fit_transform(texts).toarray()
print(tf_data.shape)
print(tf_data)

(10, 31)
[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 2 2 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1]]


The term frequency vectorizer in sklearn creates a set of words out of all the tokens, like we did above, then counts the number of times a given word occurs within a given text, returning that text as a vector. Thus, the second sentence above:

    "Ten sentence stories and ten word stories were once popular." 

which we had tokenized and stopworded to become:

    ten sentence stories ten word stories popular

becomes a list of numbers, or a vector, that looks like this:

    0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 2 2 0 0 0 0 0 0 0 1 0 0

I chose the second sentence because it has two words that occur twice, *ten* and *stories*, so that it didn't look like a line of binary. If you stack all ten texts on top of each other, then you get a matrix that of 10 rows, each row a text, and 31 columns, each column one of the important, lexical, words.

Based on the location of the two twos, my guess is that the `CountVectorizer` alphabetizes its list of words, which can also be considered as **features** of a text. A quick check of our set of words, sorted alphabetically is our first step in confirmation. (It also reveals one of the great problems of working with words: "sentence" and "sentences" as well as "word" and "words" are treated separately and so where a human being would regard those as two lexical entries, the computer treats it as four. This is one argument for stemming, but stemming, so far as I have encountered is, is not only no panacea, it also creates other problems.)

In [6]:
the_words = list(set(all_words))
the_words.sort()
print(the_words)

['able', 'comes', 'consists', 'create', 'difficult', 'fact', 'fairly', 'just', 'keep', 'limiting', 'matrix', 'need', 'popular', 'quite', 'see', 'sentence', 'sentences', 'small', 'smaller', 'stories', 'ten', 'texts', 'things', 'topics', 'total', 'used', 'variety', 'vocabulary', 'word', 'words', 'work']


We can actually get that same list from the vectorizer itself with the `get_feature_names` method:

In [7]:
features = vec.get_feature_names()
print(features)

['able', 'comes', 'consists', 'create', 'difficult', 'fact', 'fairly', 'just', 'keep', 'limiting', 'matrix', 'need', 'popular', 'quite', 'see', 'sentence', 'sentences', 'small', 'smaller', 'stories', 'ten', 'texts', 'things', 'topics', 'total', 'used', 'variety', 'vocabulary', 'word', 'words', 'work']


We can actually get the count for each term with the `vocabulary_` method, which reveals that sklearn stores the information as a dictionary with the term as the key and the count as the value:

In [8]:
occurrences = vec.vocabulary_
print(occurrences)

{'matrix': 10, 'word': 28, 'able': 0, 'ten': 20, 'see': 14, 'stories': 19, 'need': 11, 'smaller': 18, 'quite': 13, 'sentences': 16, 'variety': 26, 'create': 3, 'comes': 1, 'things': 22, 'work': 30, 'vocabulary': 27, 'popular': 12, 'texts': 21, 'difficult': 4, 'fairly': 6, 'total': 24, 'small': 17, 'words': 29, 'sentence': 15, 'just': 7, 'consists': 2, 'topics': 23, 'used': 25, 'fact': 5, 'limiting': 9, 'keep': 8}


It's also worth pointing out that we can get a count of particular terms within our corpus by feeding the `CountVectorizer` a `vocabulary` argument. Here I've prepopulated a list with three of our terms -- "sentence", "stories", and "vocabulary" -- and the function returns an array which counts only the occurrence of those three terms across all ten texts:

In [9]:
# =-=-=-=-=-=-=-=-=-=-=
# Controlled Vocabulary Count
# =-=-=-=-=-=-=-=-=-=-= 

tags = ['sentence', 'stories', 'vocabulary']
cv = CountVectorizer(vocabulary=tags)
data = cv.fit_transform(texts).toarray()
print(data)

[[0 0 0]
 [1 2 0]
 [0 0 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 0]]


So far we've been trafficking in raw counts, or occurrences, of a word -- aka term, aka feature -- in our corpus. Chances are, longer, or bigger, texts which simply have more words will have more of any given word, which means they may come to be overvalued (overweighted?) if we rely only on occurrences. Fortunately, we can simply normalize by length of a text to get a value that can be used to compare how often a word is used in relationship to the size of the text across all texts in a corpus. That is, we can get a term's **frequency**.

As I was working on this bit of code, I learned that sklearn stores this information in a compressed sparse row matrix, wherein a series of `(text, term)` coordinates are followed by a value. I have captured the first two texts below. (Note the commented out `toarray` method in the second-to-last line. It's there so often in sklearn code that I had come to take it for granted.)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=False).fit(tf_data)
words_tf = tf_transformer.transform(tf_data)#.toarray()
print(words_tf[0:2])

  (0, 2)	0.5
  (0, 16)	0.5
  (0, 20)	0.5
  (0, 29)	0.5
  (1, 12)	0.301511344578
  (1, 15)	0.301511344578
  (1, 19)	0.603022689156
  (1, 20)	0.603022689156
  (1, 28)	0.301511344578


And here's that same information represented as an array:

In [11]:
words_tf_array = words_tf.toarray()
print(words_tf_array[0:2])

[[ 0.          0.          0.5         0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.5         0.          0.          0.          0.5
   0.          0.          0.          0.          0.          0.          0.
   0.          0.5         0.        ]
 [ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.30151134
   0.          0.          0.30151134  0.          0.          0.
   0.60302269  0.60302269  0.          0.          0.          0.          0.
   0.          0.          0.30151134  0.          0.        ]]


Finally, we can also weight words within a document contra the number of times they occur within the overall corpus, thus lowering the value of common words.

In [12]:
# =-=-=-=-=-=-=-=-=-=-=
# TFIDF
# =-=-=-=-=-=-=-=-=-=-= 

tfidf = TfidfVectorizer()
tfidf_data = tfidf.fit_transform(texts)#.toarray()
print(tfidf_data.shape)
print(tfidf_data[1]) # values for second sentence

(10, 31)
  (0, 12)	0.338083066465
  (0, 28)	0.338083066465
  (0, 19)	0.67616613293
  (0, 15)	0.338083066465
  (0, 20)	0.447100526936


And now, again, in the more common form of an array:

In [13]:
tfidf_array = tfidf_data.toarray()
print(tfidf_array[1]) # values for second sentence

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.33808307
  0.          0.          0.33808307  0.          0.          0.
  0.67616613  0.44710053  0.          0.          0.          0.          0.
  0.          0.          0.33808307  0.          0.        ]


In [14]:
tfidf_feature_names = tfidf.get_feature_names()
print(tfidf_feature_names)

['able', 'comes', 'consists', 'create', 'difficult', 'fact', 'fairly', 'just', 'keep', 'limiting', 'matrix', 'need', 'popular', 'quite', 'see', 'sentence', 'sentences', 'small', 'smaller', 'stories', 'ten', 'texts', 'things', 'topics', 'total', 'used', 'variety', 'vocabulary', 'word', 'words', 'work']


Looking at our older code, it looks like the moment to populate the various facets of the vectorizer is when you are first setting it up:

    vectorizer = sk_text.TfidfVectorizer(max_df = max_percent, 
                                         min_df = min_percent,
                                         max_features = n_features,
                                         stop_words = tt_stopwords)
    tfidf = vectorizer.fit_transform(strungs)
    
We'll revisit that possibility in a moment in the next section. For now, let's just continue with the exploration of NMF. In the code below, I should note that you can feed the model either the date in the CSR matrix or the full array. The results are the same.

In [15]:
from sklearn.decomposition import NMF

topics = 2
model = NMF(n_components = topics,
          random_state = 1,
          alpha = 0.1,
          l1_ratio = 0.5).fit(tfidf_array)
print(model)

NMF(alpha=0.1, beta=1, eta=0.1, init=None, l1_ratio=0.5, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)


In [16]:
W = model.fit_transform(tfidf_array)
H = model.components_

print(W)
print(H)

[[ 0.70815391  0.        ]
 [ 0.26356173  0.        ]
 [ 0.62679093  0.13565946]
 [ 0.71158262  0.        ]
 [ 0.09659249  0.06987702]
 [ 0.          0.        ]
 [ 0.          0.53208018]
 [ 0.          0.6928437 ]
 [ 0.          0.7021627 ]
 [ 0.          0.00840084]]
[[ 0.          0.          0.25460934  0.16902145  0.33877366  0.          0.
   0.16902145  0.          0.20214662  0.          0.          0.02556778
   0.16902145  0.          0.02556778  0.38301424  0.          0.
   0.08382612  0.52326159  0.          0.          0.          0.          0.
   0.          0.08896514  0.02556778  0.46669161  0.        ]
 [ 0.          0.19739504  0.          0.          0.          0.
   0.21789602  0.          0.21789602  0.00701435  0.16441155  0.          0.
   0.          0.          0.          0.          0.57914069  0.          0.
   0.          0.          0.          0.          0.21789602  0.
   0.30019012  0.56503064  0.          0.          0.        ]]


This print/display code must be somewhere in the sklearn documentation: both [Alan Riddell][] and [Aneesha Bakharia][] use a version of it.

[Alan Riddell]: https://de.dariah.eu/tatom/topic_model_python.html
[Aneesha Bakharia]: https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730

In [17]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}: ".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(model, tfidf_feature_names, no_top_words)

Topic 0: 
ten words sentences difficult consists limiting just quite create vocabulary
Topic 1: 
small vocabulary variety total keep fairly comes matrix limiting work


Mostly what this reveals is that I need to go back and tweak my toy corpus so that the topics can be more clearly seen.

## Staying within the `sklearn` ecosystem

What if we do all tokenization and normalization in `sklearn`?

In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# This is the bog-standard version from the documentation
sk_test_vec = TfidfVectorizer(input=sentences, 
                           lowercase=True, 
                           preprocessor=None, 
                           tokenizer=None, 
                           stop_words=stopwords, 
                           token_pattern=u'(?u)\b\w\w+\b', 
                           ngram_range=(1, 1), 
                           analyzer=u'word', 
                           max_df=1.0, 
                           min_df=1, 
                           max_features=None, 
                           vocabulary=None, 
                           binary=False)

In [25]:
sk_test_data = tfidf.fit_transform(sentences)

In [26]:
topics = 2
sk_test_model = NMF(n_components = topics,
          random_state = 1,
          alpha = 0.1,
          l1_ratio = 0.5).fit(sk_test_data)
print(sk_test_model)

NMF(alpha=0.1, beta=1, eta=0.1, init=None, l1_ratio=0.5, max_iter=200,
  n_components=2, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)


In [27]:
W = sk_test_model.fit_transform(sk_test_data)
H = sk_test_model.components_

To my non-mathematical mind, it just seems pretty amazing that you can *see* so clearly the division of the larger matrix by the topics. First, we have the document-topic matrix and we can see the assignment of topics to the documents. (And this is with a terrible set of texts in which it's not clear there are any particular topics -- so, I guess, a lot like life, really.)

In [28]:
print(W)

[[ 0.          0.63355125]
 [ 0.          0.09748322]
 [ 0.14559639  0.68872325]
 [ 0.          0.87739668]
 [ 0.05021328  0.04843044]
 [ 0.          0.        ]
 [ 0.30350074  0.        ]
 [ 0.84225589  0.        ]
 [ 0.75195608  0.        ]
 [ 0.15976003  0.01560689]]


And then here's the words spread across the two topics in a `2 x 31` matrix:

In [29]:
print(H)

[[ 0.          0.          0.          0.          0.          0.
   0.03927719  0.17934791  0.          0.          0.          0.03927719
   0.          0.          0.20709477  0.          0.          0.04967716
   0.          0.          0.          0.20709477  0.00100218  0.17230381
   0.          0.03927719  0.          0.03927719  0.          0.          0.
   0.          0.          0.          0.          0.53384117  0.          0.
   0.          0.          0.23444203  0.          0.          0.
   0.16079732  0.03927719  0.          0.20709477  0.          0.03897709
   0.34046978  0.23331632  0.          0.          0.          0.14734992
   0.          0.          0.        ]
 [ 0.          0.08269729  0.          0.          0.          0.          0.
   0.          0.10244385  0.15012346  0.28401921  0.          0.10244385
   0.          0.          0.          0.          0.          0.28401921
   0.15012346  0.15012346  0.          0.15980241  0.          0.          0.

In [32]:
feature_names = sk_test_data.get_feature_names()
print(feature_names)

AttributeError: get_feature_names not found